# Changes in Land Cover: Albedo and Carbon Sequestration

**Content creators:** Oz Kira, Julius Bamah

**Content reviewers:** Yuhan Douglas Rao, Abigail Bodner

**Content editors:** Zane Mitrevica, Natalie Steinemann, Jenna Pearson, Chi Zhang, Ohad Zivan

**Production editors:**  Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS and Google DeepMind

In [ ]:
# @title Project background 
#This will be a short video introducing the content creator(s) and motivating the research direction of the template.
#The Tech team will add code to format and display the video

The global radiative budget is affected by land cover (e.g., forests, grasslands, agricultural fields, etc.), as different classifications of land cover have different levels of reflectance, or albedo. Vegetation also sequesters carbon at the same time, potentially counteracting these radiative effects.

In this project,  you will evaluate the albedo change vs. carbon sequestration. In addition, you will track significant land cover changes, specifically the creation and abandonment of agricultural land. 

In this project, you will have the opportunity to explore terrestrial remote sensing (recall our W1D3 tutorial on remote sensing) and meteorological data from GLASS and ERA5. The datasets will provide information on reflectance, albedo, meteorological variables, and land cover changes in your region of interest. We encourage you to investigate the relationships between these variables and their impact on the global radiative budget. Moreover, you can track agricultural land abandonment and analyze its potential connection to climate change. This project aligns well with the topics covered in W2D3, which you are encouraged to explore further.

# Project Template
![template](https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/projects/template-images/albedo_template_map.svg)

# Data Exploration Notebook
## Project Setup

Please run the following cells to install the necessary libarries into your Jupyter notebook!
    

In [ ]:
# google colab installs
# !pip install cartopy
# !pip install DateTime 
# !pip install matplotlib
# !pip install pyhdf
# !pip install numpy
# !pip install pandas
# !pip install modis-tools

In [ ]:
# imports
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pooch
import xarray as xr
import os

## Obtain Land Cover Data and Net Primary Production (NPP) Data from MODIS

MODIS (Moderate Resolution Imaging Spectroradiometer) is a key instrument aboard the Terra (originally known as EOS AM-1) and Aqua (originally known as EOS PM-1) satellites. Terra's orbit around the Earth is timed so that it passes from north to south across the equator in the morning, while Aqua passes south to north over the equator in the afternoon. Terra MODIS and Aqua MODIS are viewing the entire Earth's surface every 1 to 2 days, acquiring data in 36 spectral bands, or groups of wavelengths (see MODIS Technical Specifications). These data will improve your understanding of global dynamics and processes occurring on the land, in the oceans, and in the lower atmosphere. MODIS plays a vital role in the development of validated, global, interactive Earth system models able to predict global change accurately enough to assist policy makers in making sound decisions concerning the protection of our environment (https://modis.gsfc.nasa.gov/data/). The MCD12Q1.061 MODIS Land Cover Type Yearly Global 500m sinusoidal grid land cover data are available [here](https://lpdaac.usgs.gov/products/mcd12q1v061/) , and the  MOD17A3HGF.061 MODIS/Terra Net Primary Production Gap-Filled Yearly L4 Global 500m sinusoidal grid dataset is available [here](https://lpdaac.usgs.gov/products/mod17a3hgfv061/).

We have this data available for you to use in the shared projects directory, but if you want to be able to download the data and use for future work after the course you can use the code below as a guide. Before doing so you will need to register for an account on the NASA earth data portal: https://urs.earthdata.nasa.gov/users/new. Then fill in your username and password below to access the data.

In [ ]:
# import the libraries

from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler
from typing_extensions import Literal

In [ ]:
# SURAJIT: please fill in an example (the one that you would use)

# obtain Land Cover Data and Net Primary Production (NPP) Data from MODIS
username = ""  # update this line with your username 
password = ""  # update this line with your password

# connect to earthdata
session = ModisSession(username=username, password=password)

# query the MODIS catalog for collections
collection_client = CollectionApi(session=session)
collections = collection_client.query(short_name="", version="") # Collection short name + version

# query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

# filter the selected granules via spatial and temporal parameters, for example for Isreal
Israel_bbox = [] # format [x_min, y_min, x_max, y_max] # add min lon,min lat, max lon and max_lat input your preferred location
Israel_granules = granule_client.query(start_date="", end_date="", bounding_box=Israel_bbox)   #choose the start and end dates for the year-month-day you prefer

# download the granules (files are in .hdf format)
GranuleHandler.download_from_granules(Israel_granules, session, threads=-1) 

In [ ]:
# SURAJIT: please modify the code so that students could download the data, convert to xarray dataarray,
# add a useable lat lon grid to the dataset/coordinates, and concatenate all years and save as .nc.

# ideally this would loop through all years of data, but right now it is just for one file. 
# we will run this before the course so that the data is pre-processed and ready to go but we need to keep an example
# so students can use after the course

# pre-process the datafiles

from pyhdf.SD import SD, SDC

import numpy as np

file_name = 'D:/modis/modis.hdf'  #path + downloaded modis data

file = SD(file_name, SDC.READ)
file.info()
datasets_dic = file.datasets()
datasets_dic = file.datasets()
print(datasets_dic)

for idx,sds in enumerate(datasets_dic.keys()):
    print( idx,sds )
    
sds_obj = file.select('Npp_500m') # select the data variable 

data = sds_obj.get() # get sds data

print( data )
 import pprint

pprint.pprint( sds_obj.attributes() )
for key, value in sds_obj.attributes().items():
    print( key, value )
    if key == 'add_offset':
          add_offset = value  
    if key == 'scale_factor':
          scale_factor = value
data = (data - add_offset) * scale_factor
data

# plot the data
plt.imshow(data, cmap='jet')
plt.colorbar()
plt.title('Npp_500m')
plt.show()

In [ ]:
# TO ADD WHEN ON THE HUB
npp_path='~/shared/Data/Projects/...'
ds_npp =xr.open_dataset(npp_path)=
ds_npp

In [ ]:
# TO ADD WHEN ON THE HUB
lc_path='~/shared/Data/Projects/...'
ds_lc =xr.open_dataset(lc_path)=
ds_lc

## Alternative Approach: Global Land Surface Satellite (GLASS) Dataset

The Global Land Surface Satellite (GLASS) datasets primarily based on NASA’s Advanced Very High Resolution Radiometer (AVHRR) long-term data record [(LTDR)](https://ltdr.modaps.eosdis.nasa.gov) and Moderate Resolution Imaging Spectroradiometer (MODIS) data, in conjunction with other satellite data and ancillary information. 

Currently, there are more than dozens of GLASS products are officially released, including leaf area index, fraction of green vegetation coverage, gross primary production, broadband albedo, land surface temperature, evapotranspiration, and so on. 

Here we provide you the datasets of GLASS from 1982 to 2015, a 34-year long annual dynamics of global land cover (GLASS-GLC) at 5 km resolution. In this datasets, there are 7 classes, including cropland, forest, grassland, shrubland, tundra, barren land, and snow/ice. 
The annual global land cover map (5 km) is presented in a GeoTIFF file format named in the form of ‘GLASS-GLC_7classes_year’ with a WGS 84 projection. The relationship between the labels in the files and the 7 land cover classes is shown in the following table

You can refer to this [paper](https://doi.pangaea.de/10.1594/PANGAEA.913496) for detailed description of this.ts


In [ ]:
# Table 1 Classification system, with 7 land cover classes. From paper https://www.earth-syst-sci-data-discuss.net/essd-2019-23
import pandas as pd
from IPython.display import display, HTML, Markdown
# Data as list of dictionaries
classification_system = [
    {"Label": 10, "Class": "Cropland", "Subclass": "Rice paddy", "Description": ""},
    {"Label": 10, "Class": "Cropland", "Subclass": "Greenhouse", "Description": ""},
    {"Label": 10, "Class": "Cropland", "Subclass": "Other farmland", "Description": ""},
    {"Label": 10, "Class": "Cropland", "Subclass": "Orchard", "Description": ""},
    {"Label": 10, "Class": "Cropland", "Subclass": "Bare farmland", "Description": ""},
    {"Label": 20, "Class": "Forest", "Subclass": "Broadleaf, leaf-on", "Description": "Tree cover≥10%; Height>5m; For mixed leaf, neither coniferous nor broadleaf types exceed 60%"},
    {"Label": 20, "Class": "Forest", "Subclass": "Broadleaf, leaf-off", "Description": ""},
    {"Label": 20, "Class": "Forest", "Subclass": "Needle-leaf, leaf-on", "Description": ""},
    {"Label": 20, "Class": "Forest", "Subclass": "Needle-leaf, leaf-off", "Description": ""},
    {"Label": 20, "Class": "Forest", "Subclass": "Mixed leaf type, leaf-on", "Description": ""},
    {"Label": 20, "Class": "Forest", "Subclass": "Mixed leaf type, leaf-off", "Description": ""},
    {"Label": 30, "Class": "Grassland", "Subclass": "Pasture, leaf-on", "Description": "Canopy cover≥20%"},
    {"Label": 30, "Class": "Grassland", "Subclass": "Natural grassland, leaf-on", "Description": ""},
    {"Label": 30, "Class": "Grassland", "Subclass": "Grassland, leaf-off", "Description": ""},
    {"Label": 40, "Class": "Shrubland", "Subclass": "Shrub cover, leaf-on", "Description": "Canopy cover≥20%; Height<5m"},
    {"Label": 40, "Class": "Shrubland", "Subclass": "Shrub cover, leaf-off", "Description": ""},
    {"Label": 70, "Class": "Tundra", "Subclass": "Shrub and brush tundra", "Description": ""},
    {"Label": 70, "Class": "Tundra", "Subclass": "Herbaceous tundra", "Description": ""},
    {"Label": 90, "Class": "Barren land", "Subclass": "Barren land", "Description": "Vegetation cover<10%"},
    {"Label": 100, "Class": "Snow/Ice", "Subclass": "Snow", "Description": ""},
    {"Label": 100, "Class": "Snow/Ice", "Subclass": "Ice", "Description": ""},
    {"Label": 0, "Class": "No data", "Subclass": "", "Description": ""}
]

df = pd.DataFrame(classification_system)
pd.set_option('display.max_colwidth', None)
html = df.to_html(index=False)
title_md = "### Table 1 GLASS classification system with 7 land cover classes. From [this paper](https://www.earth-syst-sci-data-discuss.net/essd-2019-23)."
display(Markdown(title_md))
display(HTML(html))


In [ ]:
# source of landuse data: https://doi.pangaea.de/10.1594/PANGAEA.913496
# the folder "land-use" has the data for years 1982 to 2015. choose the years you need and change the path accordingly 
path_LandUse=os.path.expanduser('~/shared/Data/Projects/Albedo/land-use/GLASS-GLC_7classes_1982.tif')
ds_landuse=xr.open_dataset(path_LandUse).rename({'x':'longitude', 'y':'latitude'})
# ds_landuse.band_data[0,:,:].plot() # how to plot the global data
ds_landuse

## ERA5-Land Monthly Averaged Data from 1950 to Present

**[ERA5-Land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=overview)** is a reanalysis dataset that offers an enhanced resolution compared to [ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5), providing a consistent view of land variables over several decades. It is created by replaying the land component of the ECMWF ERA5 climate reanalysis, which combines model data and global observations to generate a complete and reliable dataset using the laws of physics. 

ERA5-Land focuses on the water and energy cycles at the surface level, offering a detailed record starting from 1950. The data used here is a post-processed subset of the complete ERA5-Land dataset. Monthly-mean averages have been pre-calculated to facilitate quick and convenient access to the data, particularly for applications that do not require sub-monthly fields. The native spatial resolution of the ERA5-Land reanalysis dataset is 9km on a reduced Gaussian grid (TCo1279). The data in the CDS has been regridded to a regular lat-lon grid of 0.1x0.1 degrees.

### To Calculate Albedo Using ERA5-Land
ERA5 parameter [`Forecast albedo`](https://codes.ecmwf.int/grib/param-db/?id=243) provides is the measure of the reflectivity of the Earth's surface. It is the fraction of solar (shortwave) radiation reflected by Earth's surface, across the solar spectrum, for both direct and diffuse radiation. Values are between 0 and 1. Typically, snow and ice have high reflectivity with albedo values of 0.8 and above, land has intermediate values between about 0.1 and 0.4 and the ocean has low values of 0.1 or less. Radiation from the Sun (solar, or shortwave, radiation) is partly reflected back to space by clouds and particles in the atmosphere (aerosols) and some of it is absorbed. The rest is incident on the Earth's surface, where some of it is reflected. The portion that is reflected by the Earth's surface depends on the albedo. In the ECMWF Integrated Forecasting System (IFS), a climatological background albedo (observed values averaged over a period of several years) is used, modified by the model over water, ice and snow. Albedo is often shown as a percentage (%).


In [ ]:
# link for albedo data:
albedo_path='~/shared/Data/Projects/Albedo/ERA/albedo-001.nc'
ds_albedo=xr.open_dataset(albedo_path)
ds_albedo # note the official variable name is fal (forecast albedo)

for your convience, included below are preciptation and temprature ERA5 dataset for the same times as the Albedo dataset

In [ ]:
precp_path='~/shared/Data/Projects/Albedo/ERA/precipitation-002.nc'
ds_precp=xr.open_dataset(precp_path)
ds_precp # the variable name is tp (total preciptation)


In [ ]:
tempr_path='~/shared/Data/Projects/Albedo/ERA/Temperature-003.nc'
ds_tempr=xr.open_dataset(tempr_path)
ds_tempr # the variable name is t2m (temprature at 2m)


# Further Reading
- Zhao, X., Wu, T., Wang, S., Liu, K., Yang,  J. Cropland abandonment mapping at sub-pixel scales using crop phenological information and MODIS time-series images, Computers and Electronics in Agriculture, Volume 208,
2023,107763, ISSN 0168-1699,https://doi.org/10.1016/j.compag.2023.107763

- Shani Rohatyn et al.,  Limited climate change mitigation potential through forestation of the vast dryland regions.Science377,1436-1439(2022).DOI:10.1126/science.abm9684

- Hu, Y., Hou, M., Zhao, C., Zhen, X., Yao, L., Xu, Y. Human-induced changes of surface albedo in Northern China from 1992-2012, International Journal of Applied Earth Observation and Geoinformation, Volume 79, 2019, Pages 184-191, ISSN 1569-8432, https://doi.org/10.1016/j.jag.2019.03.018

- Duveiller, G., Hooker, J. & Cescatti, A. The mark of vegetation change on Earth’s surface energy balance. Nat Commun 9, 679 (2018). https://doi.org/10.1038/s41467-017-02810-8

- Yin, H., Brandão, A., Buchner, J., Helmers, D., Iuliano, B.G., Kimambo, N.E.,  Lewińska, K.E., Razenkova, E., Rizayeva, A., Rogova, N., Spawn, S.A., Xie, Y., Radeloff, V.C. Monitoring cropland abandonment with Landsat time series, Remote Sensing of Environment, Volume 246, 2020, 111873, ISSN 0034-4257,https://doi.org/10.1016/j.rse.2020.111873

- Gupta, P., Verma, S., Bhatla, R.,Chandel, A. S., Singh, J., & Payra, S.(2020). Validation of surfacetemperature derived from MERRA‐2Reanalysis against IMD gridded data setover India.Earth and Space Science,7,e2019EA000910. https://doi.org/10.1029/2019EA000910

- Cao, Y., S. Liang, X. Chen, and T. He (2015) Assessment of Sea Ice Albedo Radiative Forcing and Feedback over the Northern Hemisphere from 1982 to 2009 Using Satellite and Reanalysis Data. J. Climate, 28, 1248–1259, https://doi.org/10.1175/JCLI-D-14-00389.1.

- Westberg, D. J., P. Stackhouse, D. B. Crawley, J. M. Hoell, W. S. Chandler, and T. Zhang (2013), An Analysis of NASA's MERRA Meteorological Data to Supplement Observational Data for Calculation of Climatic Design Conditions, ASHRAE Transactions, 119, 210-221. 
https://www.researchgate.net/profile/Drury-Crawley/publication/262069995_An_Analysis_of_NASA's_MERRA_Meteorological_Data_to_Supplement_Observational_Data_for_Calculation_of_Climatic_Design_Conditions/links/5465225f0cf2052b509f2cc0/An-Analysis-of-NASAs-MERRA-Meteorological-Data-to-Supplement-Observational-Data-for-Calculation-of-Climatic-Design-Conditions.pdf